In [103]:
# 改进版本
import random
class Card:
    """牌"""

    points_of_10 = [10, 11, 12, 13]  # 10, J, Q, K都是10点

    def __init__(self, suite, face):
        self.suite = suite
        self.face = face

    def __repr__(self):
        suites = "♠♥♣♦"
        faces = ["", "A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
        return f"{suites[self.suite.value]}{faces[self.face]}"  # 返回牌的花色和点数

    def __lt__(self, other):
        if self.suite == other.suite:
            return self.face < other.face
        return self.suite.value < other.suite.value


class Poker:
    """扑克"""

    def __init__(self):
        self.cards = [
            Card(suite, face) for suite in Suite for face in range(1, 14)
        ]  # 52张牌构成的列表
        self.current = 0  # 记录发牌位置的属性

    def shuffle(self):
        """洗牌"""
        self.current = 0  # 将发牌位置设置为0，意思是从头开始发牌
        random.shuffle(self.cards)  # 通过random模块的shuffle函数实现随机乱序

    def deal(self):
        """发牌"""
        card = self.cards[self.current]
        self.current += 1
        return card

    @property
    def has_next(self):
        """还有没有牌可以发"""
        return self.current < len(self.cards)


class Player:
    """玩家"""

    def __init__(self, name):
        self.name = name
        self.cards = []  # 玩家手上的牌
        self.total_points = 0  # 分数

    def get_one(self, card):
        """摸牌"""
        self.cards.append(card)

    def hit(self):
        """要牌"""
        print(f"{self.name}选择了要牌。")
        self.get_one()

    def stand(self):
        """停牌"""
        print(f"{self.name}选择了停牌。")

    def arrange(self):
        """整理手上的牌"""
        print("cards:", self.cards)
        self.cards.sort()

    @property
    def auto_win(self):
        if len(self.cards) == 2 and self.A_card_nums == 1:
            for card in self.cards:
                if card.face != 1 and card.face in Card.points_of_10:
                    return True
            return False

    @property
    def A_card_nums(self):
        card_a_num = len([card for card in self.cards if card.face == 1])
        return card_a_num

    def calculate_points_with_ace(self):
        """正确处理A牌的点数计算"""
        total = 0
        ace_count = 0
        
        for card in self.cards:
            if card.face == 1:  # 记录A牌的数量
                ace_count += 1
            elif card.face in [10, 11, 12, 13]:  # 10点牌
                total += 10
            else:
                total += card.face
        
        # 处理A牌：优先按11计算，必要时转为1
        # 从A牌的数量ace_count中进行迭代
        for _ in range(ace_count):
            if total + 11 <= 21:  # 先将A按照11计算，加上上面计算的total，如果小于等于21，那么可以加11，这样可以让结果尽量大
                total += 11
            else: # 否则就加1
                total += 1
        
        return total

    @property
    def calculate_total_points(self):
        return self.calculate_points_with_ace()

    def judge_win_or_lose(self):
        self.total_points = self.calculate_points_with_ace()
        print("总点数:", self.total_points)
        if self.total_points > 21:
            return "lose"
        elif self.total_points == 21:
            return "win"
        else:
            return "normal"


class Banker(Player):
    """庄家"""

    def __init__(self, name):
        super().__init__(name)
        self.hidden_card = 0
        self.show_card = 0

    def show_one_card(self):
        self.show_card = random.randint(0, 1)
        print(f"庄家 {self.name} 展示的牌是: {self.cards[self.show_card]}")

    @property
    def show_hidden_card(self):
        self.hidden_card = 1 - self.show_card
        return f"庄家 {self.name} 的暗牌是: {self.cards[self.hidden_card]}"


def get_a_card(instance, p):
    instance.get_one(p.deal())


def main():
    tag = 0  # 0:player, 1: banker
    is_player_stand = 0  # 0: 没有停牌, 1: 停牌
    poker = Poker()
    poker.shuffle()
    player = Player("黄文鑫")
    banker = Banker("黄一鸣")
    get_a_card(banker, poker)
    get_a_card(banker, poker)
    get_a_card(player, poker)
    get_a_card(player, poker)
    print(f"玩家的牌是:", player.cards)
    print("=" * 50)
    banker.show_one_card()
    if banker.auto_win:
        print(f"庄家: {banker.name}获胜!")
        return
    elif player.auto_win:
        print(f"玩家: {player.name}获胜!")
        return
    while True:
        if tag == 0 and is_player_stand == 0:
            choice = input("请输入你的选择：要牌还是停牌？(hit or stand):")
            if choice == "hit":
                get_a_card(player, poker)
                print(f"玩家的牌是:", player.cards)
                result = player.judge_win_or_lose()
                if result == "win":
                    print("玩家获胜!")
                    break
                elif result == "lose":
                    print("玩家爆牌，超过21点，玩家输了!")
                    break
                else:
                    continue
            else:
                is_player_stand = 1
            tag = 1  # 轮到庄家了
        elif tag == 1 and is_player_stand == 1:
            points = banker.calculate_total_points
            if not points >= 17:
                get_a_card(banker, poker)
                print(f"庄家的牌是:", banker.cards)
                result = banker.judge_win_or_lose()
                if result == "win":
                    print("庄家获胜!")
                    break
                elif result == "lose":
                    print("庄家爆牌，超过21点，庄家输了!")
                    break
                else:
                    continue
            else:
                print(f"玩家点数: {player.calculate_total_points}, 庄家点数: {points}")
                if points > player.calculate_total_points:
                    print("庄家点数大，庄家获胜")
                elif points < player.calculate_total_points:
                    print("玩家点数大，玩家获胜")
                else:
                    print("双方平局")
                break


if __name__ == "__main__":
    main()


玩家的牌是: [♦10, ♣10]
庄家 黄一鸣 展示的牌是: ♣J
庄家的牌是: [♣J, ♣6, ♠6]
总点数: 22
庄家爆牌，超过21点，庄家输了!


In [98]:
# 我的版本，能实现，但是有些地方需要调整，已经在上面的改进版本中调整了，它是在我的答案基础上调整的
from enum import Enum
import random


class Suite(Enum):
    """花色(枚举)"""

    SPADE, HEART, CLUB, DIAMOND = range(4)


class Card:
    """牌"""

    points_of_10 = [10, 11, 12, 13]  # 10, J, Q, K都是10点

    def __init__(self, suite, face):
        self.suite = suite
        self.face = face

    def __repr__(self):
        suites = "♠♥♣♦"
        faces = ["", "A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
        return f"{suites[self.suite.value]}{faces[self.face]}"  # 返回牌的花色和点数

    def __lt__(self, other):
        if self.suite == other.suite:
            return self.face < other.face
        return self.suite.value < other.suite.value


class Poker:
    """扑克"""

    def __init__(self):
        self.cards = [
            Card(suite, face) for suite in Suite for face in range(1, 14)
        ]  # 52张牌构成的列表
        self.current = 0  # 记录发牌位置的属性

    def shuffle(self):
        """洗牌"""
        self.current = 0  # 将发牌位置设置为0，意思是从头开始发牌
        random.shuffle(self.cards)  # 通过random模块的shuffle函数实现随机乱序

    def deal(self):
        """发牌"""
        card = self.cards[self.current]
        self.current += 1
        return card

    @property
    def has_next(self):
        """还有没有牌可以发"""
        return self.current < len(self.cards)


class Player:
    """玩家"""

    def __init__(self, name):
        self.name = name
        self.cards = []  # 玩家手上的牌
        self.total_points = 0  # 分数

    def get_one(self, card):
        """摸牌"""
        self.cards.append(card)

    def hit(self):
        """要牌"""
        print(f"{self.name}选择了要牌。")
        self.get_one()

    def stand(self):
        """停牌"""
        print(f"{self.name}选择了停牌。")

    def arrange(self):
        """整理手上的牌"""
        print("cards:", self.cards)
        self.cards.sort()

    @property
    def auto_win(self):
        if len(self.cards) == 2 and self.A_card_nums == 1:
            for card in self.cards:
                print("card is:", card)
                if card.face != 1 and card.face in Card.points_of_10:
                    print("这个card是:", card)
                    return True
                return False

    @property
    def A_card_nums(self):
        print("=" * 50)
        for card in self.cards:
            print("card.face:", card.face)
        card_a_num = len([card.face for card in self.cards if card.face == 1])
        print("有几张A:", card_a_num)
        return card_a_num

    def transform_card(self, card):
        print(f"in transform_card,card.face is: {card.face}")
        if card.face == 11 or card.face == 12 or card.face == 13:
            return 10
        else:
            return int(card.face)

    @property
    def calculate_total_points(self):
        return sum(list(map(lambda x: self.transform_card(x), self.cards)))

    def judge_win_or_lose(self):
        if len(self.cards) >= 2:
            if self.A_card_nums == len(self.cards) - 1:
                first_A_card_index = 0
                for index, card in enumerate(self.cards):
                    if card.face == 1:
                        card.face = "11"
                        first_A_card_index = index
                        break
                for card in range(first_A_card_index, len(self.cards)):
                    if card.face == 1:
                        card.face = "1"
            else:
                for card in self.cards:
                    if card.face == 1:
                        card.face = "11"
        self.total_points = sum(list(map(lambda x: self.transform_card(x), self.cards)))
        print("map后的list:", list(map(lambda x: self.transform_card(x), self.cards)))
        print("总点数:", self.total_points)
        if self.total_points > 21:
            return "lose"
        elif self.total_points == 21:
            return "win"
        else:
            return "normal"


class Banker(Player):
    """庄家"""

    def __init__(self, name):
        super().__init__(name)
        self.hidden_card = 0
        self.show_card = 0

    def show_one_card(self):
        self.show_card = random.randint(0, 1)
        print(f"庄家 {self.name} 展示的牌是: {self.cards[self.show_card]}")

    @property
    def show_hidden_card(self):
        self.hidden_card = 1 - self.show_card
        return f"庄家 {self.name} 的暗牌是: {self.cards[self.hidden_card]}"


def get_a_card(instance, p):
    instance.get_one(p.deal())


def main():
    tag = 0  # 0:player, 1: banker
    is_player_stand = 0  # 0: 没有停牌, 1: 停牌
    poker = Poker()
    poker.shuffle()
    player = Player("黄文鑫")
    banker = Banker("黄一鸣")
    get_a_card(banker, poker)
    get_a_card(banker, poker)
    get_a_card(player, poker)
    get_a_card(player, poker)
    print(f"玩家的牌是:", player.cards)
    print("=" * 50)
    banker.show_one_card()
    if banker.auto_win:
        print(f"庄家: {banker.name}获胜!")
        return
    elif player.auto_win:
        print(f"玩家: {player.name}获胜!")
        return
    while True:
        if tag == 0 and is_player_stand == 0:
            choice = input("请输入你的选择：要牌还是停牌？(hit or stand):")
            if choice == "hit":
                get_a_card(player, poker)
                print(f"玩家的牌是:", player.cards)
                result = player.judge_win_or_lose()
                if result == "win":
                    print("玩家获胜!")
                    break
                elif result == "lose":
                    print("玩家爆牌，超过21点，玩家输了!")
                    break
                else:
                    continue
            else:
                is_player_stand = 1
            tag = 1  # 轮到庄家了
        elif tag == 1 and is_player_stand == 1:
            points = banker.calculate_total_points
            if not points >= 17:
                get_a_card(banker, poker)
                print(f"庄家的牌是:", banker.cards)
                result = banker.judge_win_or_lose()
                if result == "win":
                    print("庄家获胜!")
                    break
                elif result == "lose":
                    print("庄家爆牌，超过21点，庄家输了!")
                    break
                else:
                    continue
            else:
                print(f"玩家点数: {player.calculate_total_points}, 庄家点数: {points}")
                if points > player.calculate_total_points:
                    print("庄家点数大，庄家获胜")
                elif points < player.calculate_total_points:
                    print("玩家点数大，玩家获胜")
                else:
                    print("双方平局")
                break


if __name__ == "__main__":
    main()

玩家的牌是: [♥5, ♦A]
庄家 黄一鸣 展示的牌是: ♠8
card.face: 8
card.face: 1
有几张A: 1
card is: ♠8
card.face: 5
card.face: 1
有几张A: 1
card is: ♥5
玩家的牌是: [♥5, ♦A, ♥6]
card.face: 5
card.face: 1
card.face: 6
有几张A: 1
in transform_card,card.face is: 5
in transform_card,card.face is: 11
in transform_card,card.face is: 6
in transform_card,card.face is: 5
in transform_card,card.face is: 11
in transform_card,card.face is: 6
map后的list: [5, 11, 6]
总点数: 22
玩家爆牌，超过21点，玩家输了!


True
False
